In [12]:
#import script
import sys
import re
import io
def break_into_sentences(input_file,output):
    f = io.open(input_file, 'r', encoding='utf8')
    Sentence_Size = 0
    Max_Size = 40  # Max_Size od the sentences
    paragraph = f.read()
	#remove_diacritics fct
    regex = re.compile(r'[\u064B\u064C\u064D\u064E\u064F\u0650\u0651\u0652]')
    paragraph = re.sub(regex, '', paragraph)
    #remove_urls fct
    regex = re.compile(r"(http|https|ftp)://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+")
    paragraph = re.sub(regex, '', paragraph)
    regex = re.compile(r"(\d|[\u0660\u0661\u0662\u0663\u0664\u0665\u0666\u0667\u0668\u0669])+")
    paragraph = re.sub(regex, '', paragraph)
	#remove one_character words
    regex = re.compile(r'\s.\s')
    paragraph = re.sub(regex, ' ', paragraph)
    resultFile = io.open(output, 'w',encoding='utf8')
    sentences = list()
    temp_sentence = list()
    flag = False
    for ch in paragraph.strip():
        if ch in [u'؟', u'!', u'.', u':', u'؛',u'?']:
            Sentence_Size = 0
            flag = True
        elif flag:
            sentences.append(''.join(temp_sentence).strip())
            temp_sentence = []
            flag = False
        regex = re.compile(r'[إأٱآا]')
        ch = re.sub(regex, 'ا', ch)
        regex = re.compile(r'[ئ]')
        ch = re.sub(regex, 'ى', ch)
        #remove_non_arabic_symbols fct
        ch = re.sub(r'[^\u0600-\u06FF]', ' ', ch)		
        temp_sentence.append(ch)
        if ch.isspace():
           Sentence_Size = Sentence_Size + 1
           if Sentence_Size > Max_Size:
              Sentence_Size = 0
              flag = True
              
    else:
        sentences.append(''.join(temp_sentence).strip())
        for item in sentences:
            resultFile.write("%s\n" % re.sub(' +', ' ', item))



In [14]:
break_into_sentences('input.txt',"input_segmented.txt")
break_into_sentences('text_resume.txt',"text_resume_segmented.txt")


In [15]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import math
import pandas as pd
import numpy as np


In [16]:
f=open("input_segmented", "r" , encoding="utf8")
input_segmented = f.read()
f=open("text_resume_segmented.txt", "r" , encoding="utf8")
resume_segmented = f.read()
f=open("titles.txt", "r" , encoding="utf8")
titles = f.read()
f=open("bonus.txt", "r" , encoding="utf8")
bonus = f.read()
f=open("anapho.txt", "r" , encoding="utf8")
anapho = f.read()
f=open("stigma.txt", "r" , encoding="utf8")
sigma = f.read()

In [17]:

def label(phrase):
    l={}
    resumes=resume_segmented.split("\n")
    if phrase in resumes:
        l[phrase]= 1
    else:
        l[phrase]= 0
    return l
   
def PositionPhTexte(phrase):
    l= {}
    if(input_segmented.index(phrase)+1)<=math.ceil(len(input_segmented)*1/3):
        l[phrase]=1
    elif(input_segmented.index(phrase)+1)<=math.ceil(len(input_segmented)*2/3):
        l[phrase]=2
    else:
        l[phrase]=3
    return l
def Nb_mot_titre(phrase,words):
    l={}
    mots=titles.split()
    nb=0
    for w in words:
        nb=nb+mots.count(w)
    l[phrase]=nb
    return l
def Pos_ph_texte():
    l={}
    if input_segmented.index(phrase)==0:
        l[phrase]=1
    else:
        l[phrase]=0
    return l

def Nb_exp_bonus(phrase,words):
    l ={}
    nb=0
    for w in words:
        nb=nb+bonus.count(w)
    l[phrase]=nb   
    return l
def Nb_exp_stigma(phrase,words):
    l ={}
    nb=0
    for w in words:
        nb=nb+sigma.count(w)
    l[phrase]=nb   
    return l

def Idf(phrase,words):
    l={}
    idf=0
    for w in words:
        nb=0
        for p in phrases:
            if p.find(w):
                nb=nb+1
        if nb!=0:
            idf=idf+math.log(nbr_phrase/nb)
    l[phrase]=idf
    return l
def Tf(phrase,words):
    l={}
    nb=0
    t=len(input_segmented)
    for w in words:
        nb=nb+(input_segmented.count(w)/t)
    l[phrase]=nb
    return l



In [18]:
phrases=[]
words=[]
labesl_phrase ={}
position_phrase_text = {}
number_word_title ={}
text_position = {}
bonus_phrase = {}
sigma_phrase = {}
tf = {}
idf = {}


phrases=input_segmented.split("\n")
nbr_phrase=len(phrases)
for phrase in phrases:
    words=phrase.split(" ")
    labesl_phrase.update(label(phrase))
    position_phrase_text.update(PositionPhTexte(phrase))
    number_word_title.update(Nb_mot_titre(phrase,words))
    text_position.update(PositionPhTexte(phrase))
    bonus_phrase.update(Nb_exp_bonus(phrase,words))
    sigma_phrase.update(Nb_exp_stigma(phrase,words))
    tf.update(Tf(phrase,words))
    idf.update(Idf(phrase,words))
  



In [19]:
data={"c1":list(idf.values()),"c2":list(position_phrase_text.values()),"c3":list(number_word_title.values()),"c4":list(text_position.values()),"c5":list(bonus_phrase.values()),"c6":list(sigma_phrase.values()),"c7":list(tf.values()),"label":list(labesl_phrase.values())}
df= pd.DataFrame.from_dict(data)
df.to_csv(r'data.csv',sep='\t')
X = df.drop('label', axis=1)
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)
svclassifier = SVC(kernel='linear')
svclassifier.fit(X_train, y_train)

y_pred = svclassifier.predict(X_test)

f=open("result.txt", "w" , encoding="utf8")
f.write("%s \n" %confusion_matrix(y_test,y_pred))
f.write(" %s" % classification_report(y_test,y_pred))

D:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


327

In [20]:
f=open("input.txt", "r" , encoding="utf8")
final_test = f.read()

In [21]:
svclassifier.predict(final_test)

ValueError: could not convert string to float: 'انتهاك الاتفاقات ليس استدلالاً بل وقائع يومية مشهودة ... الآمال المعلّقة على اضطلاع دول الشرق الأوسط بمعالجة مشكلاتها تذوي وتتبدد قبل انعقادها\n\n\nوضاح شرارة     الحياة     - 25/06/08\nيرصد عدد من السياسيين والمراقبين، بعضهم يتمتع بصلاحيات تقريرية، هبوب «نسمة أمل» على الشرق الأوسط (أو الأدنى)، على قول برنادر كوشنير، وزير الخارجية الفرنسي. وربما كان الأمين العام لجامعة الدول العربية، عمرو موسى، أول الراصدين والآملين. فهو ذهب، بعد ان تلا رئيس وزراء قطر ووزير خارجيتها بنود اتفاق الدوحة، الى ان الاتفاق قرينة على فاعلية السياسة العربية المشتركة إذا هي حزمت أمرها وتولته. وجمع الرئيس السوري، بشار الأسد، في أول يوم من زيارته الهند، اتفاق الدوحة نفسه والمفاوضات السورية - الإسرائيلية غير المباشرة في اسطنبول في باب واحد. وتوقع ان يعودا على المنطقة بنتائج ايجابية، قال الرئيس الرئيس السوري انه يرتقبها ويعمل في سبيلها. ولم يلبث وزير الخارجية الإسباني ان ثنى على الرأي هذا. ولم يقصره على المسألتين اللتين تشتركان في دور سورية فيهما. فهو تفاءل بالتهدئة ال